### QA 
1. input molecules graph data √
2. input correct env variable √  - btw: art is what seawater?
3. connect these data ×
只匹配环境变量 20×16 = 320 和 1023对不上（这个因为同一环境条件下对一个分子不止做了一次实验，明天再研究下能怎么匹配），我明天再改下，
还是得从表格取env_df
卡住

### 1. Load molecules graph datas from pickle

In [32]:
## Load the molecular data (.pkl) for molecules with Targeted BDE & BDFE 

import os
import pickle
import networkx as nx
from torch_geometric.utils import from_networkx

folder_path = r"C:\Users\80710\OneDrive - Imperial College London\2025 engineering\GNN molecules\graph_pickles"

# Get all .pkl file paths in the folder
pkl_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".pkl")]

# Load all graphs
graph_list = []
for file_path in pkl_files:
    with open(file_path, "rb") as file:
        graph = pickle.load(file)  # Load each .pkl graph
        graph_list.append(graph)  # Store in a list

print(f"Loaded {len(graph_list)} graphs")


Loaded 20 graphs


In [33]:
print(graph_list)

[<networkx.classes.graph.Graph object at 0x000002B4FB32BB50>, <networkx.classes.graph.Graph object at 0x000002B4D49D12E0>, <networkx.classes.graph.Graph object at 0x000002B4F6E9C0A0>, <networkx.classes.graph.Graph object at 0x000002B4F76C1A60>, <networkx.classes.graph.Graph object at 0x000002B4F7B25AF0>, <networkx.classes.graph.Graph object at 0x000002B4F7B25A90>, <networkx.classes.graph.Graph object at 0x000002B4F7BC5D60>, <networkx.classes.graph.Graph object at 0x000002B4F7BE4580>, <networkx.classes.graph.Graph object at 0x000002B4F7C2CFD0>, <networkx.classes.graph.Graph object at 0x000002B4F7C2CEB0>, <networkx.classes.graph.Graph object at 0x000002B4F7C2CDC0>, <networkx.classes.graph.Graph object at 0x000002B4F7C2CC70>, <networkx.classes.graph.Graph object at 0x000002B4F7C2CBB0>, <networkx.classes.graph.Graph object at 0x000002B4F7C2C640>, <networkx.classes.graph.Graph object at 0x000002B4F7C2C130>, <networkx.classes.graph.Graph object at 0x000002B4F7CE32E0>, <networkx.classes.graph

In [40]:
import networkx as nx
from torch_geometric.utils import from_networkx

pyg_data_list = []
for G in graph_list:
    # Convert NetworkX Graph to PyTorch Geometric Data
    data = from_networkx(G)
    pyg_data_list.append(data)


Data(edge_index=[2, 104], symbol=[48], rdkit_idx=[48], bond_index=[104], bde_pred=[104], bdfe_pred=[104], mol=<rdkit.Chem.rdchem.Mol object at 0x000002B4F7C88890>, num_nodes=48)


In [41]:
# 检查
# Now 'pyg_data_list' is a list of PyG Data objects

print(pyg_data_list[19])

Data(edge_index=[2, 104], symbol=[48], rdkit_idx=[48], bond_index=[104], bde_pred=[104], bdfe_pred=[104], mol=<rdkit.Chem.rdchem.Mol object at 0x000002B4F7C88890>, num_nodes=48)


---

## 2. Load the environmental variable

In [44]:
import pandas as pd

# Load the environmental data from Excel
env_file = r"C:\Users\80710\OneDrive - Imperial College London\2025 engineering\GNN molecules\graph_pickles\dataset02.xlsx"
env_df = pd.read_excel(env_file, engine='openpyxl')

# Select only the relevant columns for the environment
env_columns = ["temperature", "seawater", "time", "concentration", "degradation_rate"]

# Ensure all columns exist in the dataset
env_df = env_df[env_columns].copy()

# Convert categorical "seawater" to numerical (if needed)
env_df["seawater"] = env_df["seawater"].map({"sea": 1, "art": 0})  # Map "sea" → 1, "art" → 0

# Drop rows with missing values
env_df = env_df.dropna().reset_index(drop=True)

# Check if it matches the number of graphs
print(f"Loaded {len(env_df)} environment rows")
print(env_df.head())


Loaded 1023 environment rows
   temperature  seawater  time  concentration  degradation_rate
0         35.6         1    30             70          0.670914
1         35.6         1    30             70          0.680071
2         35.6         1    30             70          0.655230
3         35.6         1    30             70          0.625193
4         35.6         1    30             70          0.605853


In [45]:
print(len(pyg_data_list))
print(len(env_df)) 

20
1023


---

## 3. Use environmental variables as Transformer Positional Encoding connect

simply use nx graph is enough

### 3.1  建立Build DataLoader
PYG的Data对象代表图形，图形在节点和边缘的数量上有所不同。默认的DataLoader试图像张量一样堆叠它们，是不行滴。
 Custom Collate？

In [14]:
Mgraph = pytorch_graphs[i]
my_x = Mgraph.x

print(len(my_x))

48


In [ ]:
# 1) First, create the base PyG graphs (without env, just x/edge_index/edge_attr)
#    same code as before, but don't attach .env or .y
#    we store them in base_graphs
base_graphs = []

for i, file_path in enumerate(pkl_files):
    # your Nx -> PyG code, etc...
    pyg_graph = Data(x=..., edge_index=..., edge_attr=...)
    base_graphs.append(pyg_graph)

print("Created", len(base_graphs), "graphs without environment data.")

In [24]:


# 2) Then replicate each base graph for each environment row
expanded_graphs = []

for i, g in enumerate(base_graphs):
    for j, row in env_df.iterrows():
        # Create a new Data object that merges the graph with this env row
        env_feats = [
            row["temperature"],
            row["seawater"],
            row["time"],
            row["concentration"]
        ]
        y_value = row["degradation_rate"]
        
        new_g = Data(
            x=g.x,
            edge_index=g.edge_index,
            edge_attr=g.edge_attr,
            env=torch.tensor(env_feats, dtype=torch.float),
            y=torch.tensor([y_value], dtype=torch.float)
        )
        expanded_graphs.append(new_g)

print("Created", len(expanded_graphs), "graphs with environment + target info.")


Created 4 graphs without environment data.
Created 80 graphs with environment + target info.


### 3.2 建立 env + molecules graph层 → 建立GNN
由于PYG将多个图批量分为一个大图结构，因此我们需要data.batch来确保每个节点都会收到其相应图的正确环境嵌入
1.PYG将多个图形组合在一起→来自不同图形的节点被连接
2.data.batch tracks nodes属于哪个图。
3.我们使用env_pos_emb[data.batch]添加每个节点的正确环境效果
4.这确保每个节点都会获取其相应的环境编码

In [25]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool

class MolEnvGNN(torch.nn.Module):
    def __init__(self,
                 node_in_dim,      # dimension of x (e.g. 1 if using a simple 0/1 feature)
                 node_hidden_dim,  # hidden dimension for the GNN layers
                 env_in_dim,       # dimension of environment input (4 in your case)
                 env_hidden_dim,   # hidden dimension for environment MLP
                 output_dim=1):
        super(MolEnvGNN, self).__init__()
        
        # -- GNN layers --
        self.conv1 = GCNConv(node_in_dim, node_hidden_dim)
        self.conv2 = GCNConv(node_hidden_dim, node_hidden_dim)
        
        # -- Environment MLP --
        self.env_mlp = torch.nn.Sequential(
            torch.nn.Linear(env_in_dim, env_hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(env_hidden_dim, env_hidden_dim),
            torch.nn.ReLU()
        )
        
        # -- Final regressor --
        self.lin = torch.nn.Linear(node_hidden_dim + env_hidden_dim, output_dim)

    def forward(self, x, edge_index, batch, env):
        """
        x: Node features for all graphs in the batch
        edge_index: Edge connectivity
        batch: Assignment of each node to its graph in the batch
        env: Environment features, shape [batch_size, env_in_dim]
        """
        # 1) Node embedding via GNN
        h = self.conv1(x, edge_index)
        h = F.relu(h)
        h = self.conv2(h, edge_index)
        h = F.relu(h)
        
        # 2) Graph-level embedding
        #    We pool node embeddings for each graph in the batch
        gnn_emb = global_mean_pool(h, batch)  # shape = [batch_size, node_hidden_dim]
        
        # 3) Environment embedding
        env_emb = self.env_mlp(env)           # shape = [batch_size, env_hidden_dim]
        
        # 4) Concatenate and predict
        combined = torch.cat([gnn_emb, env_emb], dim=-1)
        out = self.lin(combined)              # shape = [batch_size, 1]
        return out.squeeze(-1)                # shape = [batch_size]


### 3.3 MSE/RMSE/R2 检验 → 测试model

In [26]:
from torch_geometric.loader import DataLoader

# 1) Create a DataLoader from expanded_graphs
#    You can shuffle for random batching
train_loader = DataLoader(expanded_graphs, batch_size=8, shuffle=True)

# 2) Instantiate the model
node_in_dim = 1      
env_in_dim = 4       # temperature, seawater, time, concentration
node_hidden_dim = 64
env_hidden_dim = 32
lr = 1e-3
num_epochs = 20

model = MolEnvGNN(node_in_dim, node_hidden_dim, env_in_dim, env_hidden_dim, output_dim=1)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = torch.nn.MSELoss()  # or L1Loss, etc.

# 3) Training loop
model.train()
for epoch in range(num_epochs):
    total_loss = 0
    for batch_data in train_loader:
        # batch_data is a 'Data' object containing the batched graphs
        # Access the fields:
        x = batch_data.x                # All node features in the batch
        edge_index = batch_data.edge_index
        batch_idx = batch_data.batch    # Tells which nodes belong to which graph
        env_feats = batch_data.env      # shape [batch_size, 4]
        y_true = batch_data.y           # shape [batch_size, 1] or [batch_size]

        optimizer.zero_grad()
        y_pred = model(x, edge_index, batch_idx, env_feats)
        # Flatten y_true if needed
        loss = criterion(y_pred, y_true.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss:.4f}")

print("Training complete.")


c:\Users\80710\anaconda3\envs\alfabet-env\lib\site-packages\torch_geometric\data\storage.py:452: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'edge_attr', 'y', 'env', 'edge_index', 'x'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


ValueError: 'GCNConv' received a tuple of node features as input while this layer does not support bipartite message passing. Please try other layers such as 'SAGEConv' or 'GraphConv' instead

In [27]:
model.eval()
with torch.no_grad():
    # Example: predict on 1 batch
    batch_data = next(iter(train_loader))
    y_pred = model(batch_data.x,
                   batch_data.edge_index,
                   batch_data.batch,
                   batch_data.env)
    print("Predictions:", y_pred)
    print("Targets:", batch_data.y.view(-1))


ValueError: 'GCNConv' received a tuple of node features as input while this layer does not support bipartite message passing. Please try other layers such as 'SAGEConv' or 'GraphConv' instead

---

In [12]:
# Suppose your environment DataFrame is called `df_env`.
# Suppose it has columns:
#  ["temperature", "seawater", "time", "concentration", "degradation_rate"]
# and each row matches one molecule in `pytorch_graphs`.

# 1. Create dataset
dataset = MoleculeEnvDataset(pyg_graphs=pytorch_graphs, env_df=df_env)

# 2. Create dataloader
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=my_collate)

# 3. Initialize model
num_node_features = 1  # if your x is [N,1], e.g. one-hot for C vs H
env_input_dim = 4       # [temperature, concentration, time, maybe 'seawater']
model = SimpleGraphModel(num_node_features=num_node_features,
                         env_input_dim=env_input_dim,
                         hidden_dim=128,
                         output_dim=1)

# 4. Train
train_model(model, dataloader, device='cpu', epochs=20)


NameError: name 'df_env' is not defined

In [11]:
train_model(model, dataloader, epochs=20)

NameError: name 'model' is not defined

## 4. fixed code

In [34]:
print(len(pytorch_graphs))   # Should be 20
print(len(env_df))           # Must also be 20


20
20


In [35]:
import torch
from torch_geometric.data import Data, Dataset
from torch.utils.data import DataLoader

class MoleculeEnvDataset(Dataset):
    def __init__(self, pyg_graphs, env_df):
        """
        Args:
            pyg_graphs: list of PyG Data objects (molecules)
            env_df: pandas DataFrame with environmental conditions & target
        """
        self.pyg_graphs = pyg_graphs
        self.env_df = env_df
        
        # 1. Expand molecule data to match env_df length
        self.graphs_expanded = [pyg_graphs[i % len(pyg_graphs)] for i in range(len(env_df))]

        # 2. Extract environmental conditions
        self.env_features = torch.tensor(
            self.env_df[["temperature", "seawater", "time", "concentration"]].values, dtype=torch.float
        )

        # 3. Extract degradation rate as target
        self.targets = torch.tensor(self.env_df["degradation_rate"].values, dtype=torch.float)

    def __len__(self):
        return len(self.env_df)  # Matches number of experimental conditions

    def __getitem__(self, idx):
        graph = self.graphs_expanded[idx]  # Get corresponding molecule
        env_features = self.env_features[idx]  # Get environmental variables
        target = self.targets[idx]  # Get degradation rate

        return graph, env_features, target


In [36]:
# 创建 Dataset
dataset = MoleculeEnvDataset(pyg_graphs=pytorch_graphs, env_df=env_df)

# 创建 DataLoader
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# 检查 Dataset 结构
graph, env_features, target = dataset[0]
print(graph)  # Should be a PyG Data object
print(env_features)  # Should be a tensor with 4 environmental variables
print(target)  # Should be a single float value


Data(x=[42, 1], edge_index=[2, 44], edge_attr=[44, 2])
tensor([35.6000,  1.0000, 30.0000, 70.0000])
tensor(0.6709)


In [37]:
# 假设你的 GNN 结构如下：
num_node_features = 1  # 节点特征
env_input_dim = 4  # 环境变量：[温度，海水类型，时间，浓度]
model = SimpleGraphModel(num_node_features=num_node_features,
                         env_input_dim=env_input_dim,
                         hidden_dim=128,
                         output_dim=1)

# 训练模型
train_model(model, dataloader, device='cpu', epochs=20)


NameError: name 'SimpleGraphModel' is not defined

---

In [ ]:
import itertools
import torch
import pandas as pd

# 所有环境变量的可能取值
temperature_values = [35.6, 86.0]
seawater_values = [0, 1]  # 这里用 0 (sea), 1 (art) 代替字符串
time_values = [30, 60]
concentration_values = [70, 700]

# **创建所有组合（16个环境变量组合）**
env_conditions = list(itertools.product(
    temperature_values, seawater_values, time_values, concentration_values
))

# 转为 Pandas DataFrame，便于查看
env_df = pd.DataFrame(env_conditions, columns=["temperature", "seawater", "time", "concentration"])
print(env_df)


    temperature  seawater  time  concentration
0          35.6         0    30             70
1          35.6         0    30            700
2          35.6         0    60             70
3          35.6         0    60            700
4          35.6         1    30             70
5          35.6         1    30            700
6          35.6         1    60             70
7          35.6         1    60            700
8          86.0         0    30             70
9          86.0         0    30            700
10         86.0         0    60             70
11         86.0         0    60            700
12         86.0         1    30             70
13         86.0         1    30            700
14         86.0         1    60             70
15         86.0         1    60            700


In [ ]:
from torch_geometric.data import Dataset, Data, Batch
from torch.utils.data import DataLoader

class MoleculeEnvDataset(Dataset):
    def __init__(self, pyg_graphs, env_df):
        """
        Args:
            pyg_graphs: list of 20 PyG 分子图
            env_df: 16 组环境变量
        """
        self.pyg_graphs = pyg_graphs  # 20个分子
        self.env_df = env_df

        # **创建 (20 × 16) 组实验数据**
        self.dataset = [
            (self.pyg_graphs[i % len(self.pyg_graphs)], torch.tensor(env, dtype=torch.float))
            for i in range(len(self.pyg_graphs) * len(self.env_df))
            for env in self.env_df.values
        ]

    def __len__(self):
        return len(self.dataset)  # 320 组数据

    def __getitem__(self, idx):
        graph, env_features = self.dataset[idx]
        return graph, env_features

# **创建 Dataset**
dataset = MoleculeEnvDataset(pyg_graphs=pytorch_graphs, env_df=env_df)


In [ ]:
class MoleculeEnvDataset(Dataset):
    def __init__(self, pyg_graphs, env_df, target_values):
        """
        Args:
            pyg_graphs: list of 20 PyG 分子图
            env_df: 16 组环境变量
            target_values: 320 组目标降解率 (每个分子在不同环境下的实验结果)
        """
        self.pyg_graphs = pyg_graphs
        self.env_df = env_df
        self.target_values = target_values  # 目标值 (320个)

        # **创建 (20 × 16) 组实验数据**
        self.dataset = [
            (self.pyg_graphs[i % len(self.pyg_graphs)],  # Graph
             torch.tensor(env, dtype=torch.float),       # 环境变量
             torch.tensor(self.target_values[i], dtype=torch.float))  # 目标值
            for i in range(len(self.pyg_graphs) * len(self.env_df))
            for env in self.env_df.values
        ]

    def __len__(self):
        return len(self.dataset)  # 320 组数据

    def __getitem__(self, idx):
        graph, env_features, target = self.dataset[idx]
        return graph, env_features, target


In [ ]:
def my_collate(batch):
    graphs, env_features, targets = zip(*batch)  # 拆分 Graph, 环境变量, 目标值
    graphs_batch = Batch.from_data_list(graphs)  # PyG 批量处理
    env_features_batch = torch.stack(env_features)  # 转 Tensor
    targets_batch = torch.stack(targets)  # 转 Tensor
    return graphs_batch, env_features_batch, targets_batch

# **创建 DataLoader**
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=my_collate)

# **检查 DataLoader**
graphs_batch, env_features_batch, targets_batch = next(iter(dataloader))
print(graphs_batch)  # PyG DataBatch
print(env_features_batch.shape)  # (batch_size, 4)
print(targets_batch.shape)  # (batch_size,)


ValueError: not enough values to unpack (expected 3, got 2)

In [ ]:
# 假设 GNN 结构如下：
num_node_features = 1  # 节点特征
env_input_dim = 4  # 环境变量：[温度，海水类型，时间，浓度]

model = SimpleGraphModel(num_node_features=num_node_features,
                         env_input_dim=env_input_dim,
                         hidden_dim=128,
                         output_dim=1)

# **训练模型**
train_model(model, dataloader, device='cpu', epochs=20)


ValueError: not enough values to unpack (expected 3, got 2)